In [ ]:
import numpy as np
import pandas as pd

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [ ]:
data = pd.read_csv("../data/adult.csv")
data = data[~data.isin(["?"]).any(axis=1)]
data.head(50)

In [ ]:
check_data_type = data[['salary', 'salary K$']].dtypes
check_isnull_salary = data['salary'].isnull().sum()
check_isnull_salary_K = data['salary K$'].isnull().sum()
print(check_data_type, check_isnull_salary, check_isnull_salary_K )

# Task 1
Print the count of men and women in the dataset.

In [ ]:
data[data["sex"].isin(["Male", "Female"])].shape

# Task 2
Find the average age of men in dataset

In [ ]:
data[data["sex"] == "Male"]["age"].mean()

# Task 3
Get the percentage of people from Poland (native-country)

In [ ]:
total_count = data.shape[0]
poland_count = data[data["native-country"] == "Poland"].shape[0]
(poland_count / total_count) * 100

Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [ ]:
people_more_50K_mean = data[data["salary"] == ">50K"]["age"].mean()
people_less_50K_mean = data[data["salary"] == "<=50K"]["age"].mean()
print(people_more_50K_mean, people_less_50K_mean)
people_more_50K_std = data[data["salary"] == ">50K"]["age"].std()
people_less_50K_std = data[data["salary"] == "<=50K"]["age"].std()
print(people_more_50K_std, people_less_50K_std)

# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [ ]:
data[data["salary"] == ">50K"].isin(["Bachelors", "Prof-school", "Assoc-acdm", "Assoc-voc", "Masters", "Doctorate"]).shape[0]

# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [ ]:
data.groupby("education").describe()


# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [ ]:
men = data[data["sex"] == "Male"]

married_men = men[men["marital-status"].str.startswith("Married")]
non_married_men = men[~men["marital-status"].str.startswith("Married")]

married_more_50k = married_men[married_men["salary"] == ">50K"]["salary"].count()
married_less_50k = married_men[married_men["salary"] == "<=50K"]["salary"].count()

non_married_more_50k = non_married_men[non_married_men["salary"] == ">50K"]["salary"].count()
non_married_less_50k = non_married_men[non_married_men["salary"] == "<=50K"]["salary"].count()

print(married_more_50k, non_married_more_50k)
print(married_less_50k, non_married_less_50k)
print(non_married_more_50k, non_married_less_50k)

# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [ ]:
max_hours = data["hours-per-week"].max()
data[data["hours-per-week"] == max_hours].shape[0]

# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [ ]:
data.corr()